In [1]:
# k = 10
output_files = ['valid_2000_dir_k10_gpt4.json', 'valid_2000_dir_k10_gpt35.json',
                'valid_2000_tax_5feats_k10_gpt4.json', 'valid_2000_tax_5feats_k10_gpt35.json',
                'valid_2000_tax_20feats_k10_gpt4.json', 'valid_2000_tax_20feats_k10_gpt35.json']

direct_rec_raw_outputs = ['valid_2000_dir_k10_gpt4.json', 
                          'valid_2000_dir_k10_gpt35.json']

tax_rec_raw_outputs = ['valid_2000_tax_5feats_k10_gpt4.json', 'valid_2000_tax_5feats_k10_gpt35.json',
                       'valid_2000_tax_20feats_k10_gpt4.json', 'valid_2000_tax_20feats_k10_gpt35.json']

# k = 1
tax_k1_raw = ['valid_2000_tax_5feats_k1_gpt4.json', 'valid_2000_tax_5feats_k1_gpt35.json',
              'valid_2000_tax_20feats_k1_gpt4.json', 'valid_2000_tax_20feats_k1_gpt35.json']

dir_k1_raw = ['valid_2000_dir_k1_gpt4.json', 
              'valid_2000_dir_k1_gpt35.json']

# k = 1, plus movie name
tax_name_k1_raw = ['valid_2000_tax_5feats_name_k1_gpt4.json', 'valid_2000_tax_5feats_name_k1_gpt35.json',
                   'valid_2000_tax_20feats_name_k1_gpt4.json', 'valid_2000_tax_20feats_name_k1_gpt35.json']

### For dir

In [2]:
# transform raw output to movies for direct recommendation
import json

directory = 'ml-100k/'
for output_file in dir_k1_raw:
    with open(directory + output_file, 'r') as file:
        generate = json.load(file)
    
    # next(iter(eval(generate[0]['output']).values()))
    for index, entry in enumerate(generate):
        # print(index)
        output = entry['output'].strip('```').strip('json').replace('\"[', '[').replace(']\"', ']').replace('\\', '')
        entry['rec_movies'] = next(iter(eval(output).values()))
    
    with open(directory + output_file.strip('.json') + '_processed.json', 'w') as file:
        json.dump(generate, file, indent=4)


In [3]:
# for direct recommendation
# add top-10 similar movies for each top-1 recommendation to the processed outputs

import json
from rouge_score import rouge_scorer


# Calculate the Rouge-L score between the target movie and all candidate movies
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) # Use Rouge-L scorer to calculate the similarity between the target movie and the candidate movies

def calculate_rougeL_score(target, candidates):
    scores = []
    for candidate in candidates:
        score = scorer.score(target, candidate)['rougeL'].fmeasure
        scores.append((candidate, score))
    return scores

def get_top_k_similar_movies(target_movie, candidate_movies, k=10):
    # Calculate the Rouge-L score between the target movie and all candidate movies
    scores = calculate_rougeL_score(target_movie, candidate_movies)

    # Sort the movies based on Rouge-L score in descending order and select the top k
    top_k_similar_movies = sorted(scores, key=lambda x: x[1], reverse=True)[:k]

    # Initialize two lists to store the top-k movies and their scores
    movies_list = [movie for movie, _ in top_k_similar_movies]
    scores_list = [score for _, score in top_k_similar_movies]

    return movies_list, scores_list

directory = 'ml-100k/'
direct_rec_processed = ['valid_2000_dir_k1_gpt4_processed.json', 'valid_2000_dir_k1_gpt35_processed.json']

with open(directory + "movie_taxonomy_5features.json", "r") as file:
    movies_dict = json.load(file)
all_candidate_movies = list(movies_dict.keys())


for processed_file in direct_rec_processed:
    with open(directory + processed_file, 'r') as file:
        generate = json.load(file)
        
    # next(iter(eval(generate[0]['output']).values()))[0]
    for index, entry in enumerate(generate):
        # print(index)
        output = entry['output'].strip('```').strip('json').replace('\"[', '[').replace(']\"', ']').replace('\\', '')
        top1_rec_movie = next(iter(eval(output).values()))[0]
        top10_rec_movies, top10_scores = get_top_k_similar_movies(top1_rec_movie, all_candidate_movies, k=10)
        entry['top10_similar_for_top1'] = top10_rec_movies
        entry['top10_scores'] = top10_scores

    # generate[0]['top10_similar_for_top1']
    with open(directory + processed_file, 'w') as file:
        json.dump(generate, file, indent=4)

### For tax

In [12]:
import json

tax_rec_raw_outputs = ['valid_2000_tax_5feats_k10_gpt4.json', 'valid_2000_tax_5feats_k10_gpt35.json',
                       'valid_2000_tax_20feats_k10_gpt4.json', 'valid_2000_tax_20feats_k10_gpt35.json']

tax_k1_raw = ['valid_2000_tax_5feats_k1_gpt4.json', 'valid_2000_tax_5feats_k1_gpt35.json',
            'valid_2000_tax_20feats_k1_gpt4.json', 'valid_2000_tax_20feats_k1_gpt35.json']

tax_name_k1_raw = ['valid_2000_tax_5feats_name_k1_gpt4.json', 'valid_2000_tax_5feats_name_k1_gpt35.json',
                   'valid_2000_tax_20feats_name_k1_gpt35.json', 'valid_2000_tax_20feats_name_k1_gpt4.json']

directory = 'ml-100k/'
with open(directory + 'valid_2000_tax_20feats_name_k1_gpt4.json', 'r') as file:
    generate = json.load(file)

eval(generate[0]['output'])
# eval(generate[0]['output'].replace('"\n', '\n').replace(':', '":'))
# top1_rec_movie = eval(generate[0]['output'].replace('\"\n', '\n').replace('):', ')":'))
# top1_rec_movie

{'The Shawshank Redemption (1994)': ['Drama',
  'Survival',
  'Unknown',
  '9+',
  '90%+',
  '80+',
  'R',
  'Critically Acclaimed',
  'Long (> 120 minutes)',
  'Live Action',
  '2D',
  'English',
  'United States',
  '1990s',
  'Oscar',
  'Medium Budget',
  'Successful',
  'Iconic',
  'Based on a Book',
  'None']}

In [13]:
n_error = 0
error_index = []
for index, entry in enumerate(generate):
    try:
        # eval(entry['output'].replace('\"\n', '\n').replace('):', ')":')) # for other tax files
        eval(entry['output'].replace('```', '')) # for 'valid_2000_tax_20feats_name_k1_gpt4.json'
    except:
        try:
            eval('{'+entry['output'].replace('```', '')+'}')
        except:
            n_error += 1
            print(index)
            error_index.append(index)
            print(entry['output'])

n_error

0

In [67]:
tax_name_k1_raw = ['valid_2000_tax_5feats_name_k1_gpt4.json', 'valid_2000_tax_5feats_name_k1_gpt35.json',
                   'valid_2000_tax_20feats_name_k1_gpt35.json', 'valid_2000_tax_20feats_name_k1_gpt4.json']

directory = 'ml-100k/'
with open(directory + 'valid_2000_tax_5feats_name_k1_gpt4.json', 'r') as file:
    generate = json.load(file)

# for other generate files
for entry in generate:
    try:
        output = eval(entry['output'].replace('\"\n', '\n').replace('):', ')":'))
    except:
        print(entry['output'])

    if isinstance(output, dict):
        processed_output = list(output.keys())
        rec_tax = next(iter(output.values()))
        if list_dimensions(rec_tax) == 1:
            processed_output.append(rec_tax)
            # print(processed_output)
        elif list_dimensions(rec_tax) == 2:
            flatten_rec_tax = [item for sublist in rec_tax for item in sublist]
            processed_output.append(flatten_rec_tax)
            # print(processed_output)
    
    # elif isinstance(output, list):
    #     # entry['rec_movies'] = output

    #     # rec_movie_name = [output[0]]
    #     # rec_tax = output[1]
    #     processed_output = output
    #     print(processed_output)
    
    # else:
    #     print("not a dict or list")

In [54]:
# for 'valid_2000_tax_20feats_name_k1_gpt4.json'

def list_dimensions(lst):
    """递归函数，用以确定列表的维度。"""
    if not isinstance(lst, list):
        return 0
    if len(lst) == 0:
        return 1
    return 1 + max(list_dimensions(item) for item in lst)

for entry in generate:
    try:
        output = eval(entry['output'].replace('```', ''))
    except:
        output = eval('{'+entry['output'].replace('```', '')+'}')

    if isinstance(output, dict):
        # entry['rec_movies'] = next(iter(output.values()))
        # concate the key and values to a list

        # print(list(output.keys())[0])
        # print(next(iter(output.values())))
        # rec_movie_name = list(output.keys())
        processed_output = list(output.keys())
        rec_tax = next(iter(output.values()))
        if list_dimensions(rec_tax) == 1:
            processed_output.append(rec_tax)
            print(processed_output)
        elif list_dimensions(rec_tax) == 2:
            flatten_rec_tax = [item for sublist in rec_tax for item in sublist]
            processed_output.append(flatten_rec_tax)
            print(processed_output)

        
    elif isinstance(output, list):
        # entry['rec_movies'] = output

        # rec_movie_name = [output[0]]
        # rec_tax = output[1]
        processed_output = output
        print(processed_output)


['The Shawshank Redemption (1994)', ['Drama', 'Survival', 'Unknown', '9+', '90%+', '80+', 'R', 'Critically Acclaimed', 'Long (> 120 minutes)', 'Live Action', '2D', 'English', 'United States', '1990s', 'Oscar', 'Medium Budget', 'Successful', 'Iconic', 'Based on a Book', 'None']]
['The Shawshank Redemption (1994)', ['Drama', 'Justice', 'Unknown', '9+', '90%+', '80+', 'R', 'Critically Acclaimed', 'Long (> 120 minutes)', 'Color', '2D', 'English', 'United States', '1990s', 'Oscar', 'Medium Budget', 'Successful', 'Iconic', 'Based on a Book', 'Violence']]
['The Shawshank Redemption (1994)', ['Drama', 'Friendship', 'Netflix', '9+', '90%+', '80+', 'R', 'Critically Acclaimed', 'Long (> 120 minutes)', 'Live Action', '2D', 'English', 'United States', '1990s', 'Oscar', 'Medium Budget', 'Successful', 'Iconic', 'Based on a Book', 'Violence']]
['The Shawshank Redemption (1994)', ['Drama', 'Justice', 'Other', '9+', '90%+', '80+', 'R', 'Critically Acclaimed', 'Long (> 120 minutes)', 'Color', '2D', 'Engl

In [49]:
type(rec_tax)

list

In [26]:
for index in error_index:
    whole_output = generate[index]['output']
    # Find the indices for the first occurrence of "[[" and "]]"
    start_index = whole_output.find("[[")
    end_index = whole_output.find("]]") + 2  # +2 to include the closing brackets
    # Extract the content
    content_str = whole_output[start_index+1:end_index]
    try:
        top1_rec = eval(content_str)
    except:
        try:
            top1_rec = eval(whole_output[start_index:end_index])
        except:
            print(index)
            print(whole_output)
            print(content_str)


57
[[['Romance'], ['Social Commentary'], ['Netflix'], ['Below 6'], ['90%+'], ['90+'], ['PG'], ['Korean'], ['2000s'], ['Critically Acclaimed'], ['United States'], ['Variable (Anthology or Series)'], ['BAFTA'], ['Live Action'], ['4DX'], ['Based on a Book'],...mation'], ['Mixed Media'], ['Black and White'], ['Color']], [['Action'], ['Adventure'], ['Hulu'], ['6+'], ['Not Applicable'], ['Not Applicable'], ['PG-13'], ['French'], ['1990s'], ['Poorly Received'], ['India'], ['Variable (Anthology or Series)'], ['BAFTA...   ['Adventure'], ['Hulu'], ['Below 6'], ['Below 60%'], ['50+'], ['Not Applicable'], ['Spanish'], ['2010s'], ['Critically Acclaimed'], ['Italy'], ['Long (> 120 minutes)'], ['Cannes'], ['Color'], ['2D'], ['Franchise'], ['Sexual Content'], ['Medium Budget']]]
[['Romance'], ['Social Commentary'], ['Netflix'], ['Below 6'], ['90%+'], ['90+'], ['PG'], ['Korean'], ['2000s'], ['Critically Acclaimed'], ['United States'], ['Variable (Anthology or Series)'], ['BAFTA'], ['Live Action'], ['4D

In [58]:
top1_2d = eval(generate[0]['output'])[0]

[item for sublist in [['No recommendation']] for item in sublist]


['No recommendation']

In [68]:
import re

def flatten_movie_dict(nested_list):
    """递归地将嵌套列表平铺为一维列表"""
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_movie_dict(item))
        else:
            flat_list.append(item)
    return flat_list

def flatten_list(original_list):
    """递归地将嵌套列表平铺为一维列表"""
    flat_list = [[item for sublist in l for item in sublist] for l in original_list]
    return flat_list

def extract_rec_type(data):
    for index, entry in enumerate(data):
        # print(index)
        text = entry['output']

        try:
            # 尝试评估entry['output']中的表达式
            entry['output'] = eval(entry['output']) # !!!!!!!!!!!!!!!!!!!!!!!!!!
        except:
            # 如果出现任何错误，执行备用方案
            entry['output'] = [['No recommendation']] * 10
        entry['output'] = flatten_list(entry['output'])
    
    return data

def extract_rec_type_top1(data):
    for index, entry in enumerate(generate):
        try:
            top1_rec_movie = eval(entry['output'])[0]
        except:
            whole_output = entry['output']
            start_index = whole_output.find("[[")
            end_index = whole_output.find("]]") + 2  # +2 to include the closing brackets
            content_str = whole_output[start_index+1:end_index]
            try:
                top1_rec_movie = eval(content_str)
            except:
                try:
                    top1_rec_movie = eval(whole_output[start_index:end_index])
                except:
                    top1_rec_movie = [['No recommendation']]
        entry['top1_movie'] = [item for sublist in top1_rec_movie for item in sublist] # transform 2D list to 1D list
        
    return data


def flatten_movies_dict(movies_dict):
    for movie, movie_tax in movies_dict.items():
        # try:
        #     temp_dict = eval(movie_tax)
        # except:
        #     print(movie)
        temp_dict = eval(movie_tax)
        value_list = []
        for value in temp_dict.values():
            value_list.append(value)
        value_list = flatten_movie_dict(value_list)
        movies_dict[movie] = value_list
    
    return movies_dict

In [ ]:
# tax_rec_raw_outputs = ['valid_2000_tax_5feats_k10_gpt4.json', 
#                        'valid_2000_tax_5feats_k10_gpt35.json',
#                        'valid_2000_tax_20feats_k10_gpt4.json', 
#                        'valid_2000_tax_20feats_k10_gpt35.json']
# movie_dicts = ['movie_taxonomy_5features.json', 
#                'movie_taxonomy_20features.json']


movie_dicts = ['movie_taxonomy_5feats_temp.json', 
               'movie_taxonomy_10feats_temp.json',
               'movie_taxonomy_20feats_temp.json']


In [16]:
directory = 'ml-100k/'

with open(directory + "movie_taxonomy_5feats_temp.json", "r") as file:
    movies_dict = json.load(file)
# all_candidate_movies = list(movies_dict.keys())
for key, value in movies_dict.items():
    # print(value.strip('```json\n').strip('\n```'))
    processed_value = value.strip('```json\n').strip('\n```')
    movies_dict[key] = processed_value
    
with open(directory + "movie_taxonomy_5feats.json", 'w') as file:
    json.dump(movies_dict, file, indent=4)

# for processed_file in direct_rec_processed:
#     with open(directory + processed_file, 'r') as file:
#         generate = json.load(file)
        
#     # next(iter(eval(generate[0]['output']).values()))[0]
#     for index, entry in enumerate(generate):
#         # print(index)
#         output = entry['output'].strip('```').strip('json').replace('\"[', '[').replace(']\"', ']').replace('\\', '')
#         top1_rec_movie = next(iter(eval(output).values()))[0]
#         top10_rec_movies, top10_scores = get_top_k_similar_movies(top1_rec_movie, all_candidate_movies, k=10)
#         entry['top10_similar_for_top1'] = top10_rec_movies
#         entry['top10_scores'] = top10_scores

#     # generate[0]['top10_similar_for_top1']
#     with open(directory + processed_file, 'w') as file:
#         json.dump(generate, file, indent=4)

In [20]:
def calculate_similarity(target_taxonomy, movie_taxonomy):
    """Calculate the similarity as the count of common words."""
    return len(set(target_taxonomy) & set(movie_taxonomy))

def find_top_k_similar_movies(movie_dict, target_movie_taxonomy, k):
    """Find the top-K similar movies based on taxonomy similarity."""
    # Calculate similarity scores for each movie in the dictionary
    similarity_scores = []
    for movie_name, movie_taxonomy in movie_dict.items():
        similarity = calculate_similarity(target_movie_taxonomy, movie_taxonomy)
        similarity_scores.append((movie_name, similarity))

    # Sort the movies based on similarity score in descending order and select the top-K
    top_k_similar_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]

    movies_list = [movie for movie, _ in top_k_similar_movies]
    scores_list = [score for _, score in top_k_similar_movies]

    return movies_list, scores_list

import json

directory = 'ml-100k/'

tax_k1_raw = ['valid_2000_tax_5feats_k1_gpt4.json', 'valid_2000_tax_5feats_k1_gpt35.json',
            'valid_2000_tax_20feats_k1_gpt4.json', 'valid_2000_tax_20feats_k1_gpt35.json']

movie_dicts = ['movie_taxonomy_5feats_temp.json', 
               'movie_taxonomy_20feats_temp.json']


with open(directory + 'valid_2000_tax_20feats_k1_gpt4.json', 'r') as file:
    generate = json.load(file)

# tax_generate = extract_rec_type(generate)
with open(directory + "movie_taxonomy_20feats.json", "r") as file:
    movies_dict = json.load(file)


movies_dict_flatten = flatten_movies_dict(movies_dict)
generate_flatten = extract_rec_type_top1(generate)

for entry in generate_flatten:
    top1_rec_tax = entry['top1_movie']
    top10_rec_movies, top_10_scores = find_top_k_similar_movies(movies_dict_flatten, top1_rec_tax, k=10)
    entry['top10_similar_for_top1'] = top10_rec_movies
    entry['top10_scores'] = top_10_scores

with open(directory + 'valid_2000_tax_20feats_k1_gpt4_processed.json', 'w') as file:
    json.dump(generate_flatten, file, indent=2)

In [73]:
# for rec with name

import json
from rouge_score import rouge_scorer

# Calculate the Rouge-L score between the target movie and all candidate movies
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) # Use Rouge-L scorer to calculate the similarity between the target movie and the candidate movies

def list_dimensions(lst):
    """递归函数，用以确定列表的维度。"""
    if not isinstance(lst, list):
        return 0
    if len(lst) == 0:
        return 1
    return 1 + max(list_dimensions(item) for item in lst)

def calculate_tax_similarity(target_taxonomy, movie_taxonomy):
    """Calculate the similarity as the count of common words."""
    return len(set(target_taxonomy) & set(movie_taxonomy))/len(set(target_taxonomy))

def find_top_k_similar_movies_name(target_movie, movie_dict, k):
    """Find the top-K similar movies based on taxonomy similarity."""
    # target_movie = [movie_name, movie_taxonomy]
    # movie_taxonomy = [feature1, feature2, ...]
    # Calculate similarity scores for each movie in the dictionary
    target_movie_name = target_movie[0]
    target_movie_taxonomy = target_movie[1]  # a list of features values
    similarity_scores = []
    for movie_name, movie_taxonomy in movie_dict.items():
        name_score = scorer.score(target_movie_name, movie_name)['rougeL'].fmeasure
        tax_score = calculate_tax_similarity(target_movie_taxonomy, movie_taxonomy)
        similarity = name_score + tax_score
        # similarity = calculate_tax_similarity(target_movie_taxonomy, movie_taxonomy)
        similarity_scores.append((movie_name, similarity))

    # Sort the movies based on similarity score in descending order and select the top-K
    top_k_similar_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]

    movies_list = [movie for movie, _ in top_k_similar_movies]
    scores_list = [score for _, score in top_k_similar_movies]

    return movies_list, scores_list

def process_generate_name(generate_file_name, generate_data):
    if generate_file_name == 'valid_2000_tax_20feats_name_k1_gpt4.json':
        for entry in generate_data:
            try:
                output = eval(entry['output'].replace('```', ''))
            except:
                output = eval('{'+entry['output'].replace('```', '')+'}')

            if isinstance(output, dict):
                processed_output = list(output.keys())
                rec_tax = next(iter(output.values()))
                if list_dimensions(rec_tax) == 1:
                    processed_output.append(rec_tax)
                elif list_dimensions(rec_tax) == 2:
                    flatten_rec_tax = [item for sublist in rec_tax for item in sublist]
                    processed_output.append(flatten_rec_tax)
                
            elif isinstance(output, list):
                processed_output = output
            
            entry['top1_movie'] = processed_output

    else:
        for entry in generate_data:
            try:
                output = eval(entry['output'].replace('\"\n', '\n').replace('):', ')":'))
            except:
                print(entry['output'])

            if isinstance(output, dict):
                processed_output = list(output.keys())
                rec_tax = next(iter(output.values()))
                if list_dimensions(rec_tax) == 1:
                    processed_output.append(rec_tax)
                elif list_dimensions(rec_tax) == 2:
                    flatten_rec_tax = [item for sublist in rec_tax for item in sublist]
                    processed_output.append(flatten_rec_tax)
            entry['top1_movie'] = processed_output
    return generate_data

directory = 'ml-100k/'
tax_name_k1_raw = ['valid_2000_tax_5feats_name_k1_gpt4.json', 'valid_2000_tax_5feats_name_k1_gpt35.json',
                   'valid_2000_tax_20feats_name_k1_gpt35.json', 'valid_2000_tax_20feats_name_k1_gpt4.json']

movie_dicts = ['movie_taxonomy_5feats.json', 
               'movie_taxonomy_20feats.json']

generate_file_name = 'valid_2000_tax_5feats_name_k1_gpt35.json'
with open(directory + generate_file_name, 'r') as file:
    generate = json.load(file)

with open(directory + "movie_taxonomy_5feats.json", "r") as file:
    movies_dict = json.load(file)

movies_dict_flatten = flatten_movies_dict(movies_dict)
generate_processed = process_generate_name(generate_file_name, generate)

for entry in generate_processed:
    top1_rec_movie = entry['top1_movie']
    top10_rec_movies, top10_scores = find_top_k_similar_movies_name(top1_rec_movie, movies_dict_flatten, k=10)
    entry['top10_similar_for_top1'] = top10_rec_movies
    entry['top10_scores'] = top10_scores

with open(directory + 'valid_2000_tax_5feats_name_k1_gpt35_processed.json', 'w') as file:
    json.dump(generate_processed, file, indent=2)

In [ ]:
# for direct recommendation
# add top-10 similar movies for each top-1 recommendation to the processed outputs

import json
from rouge_score import rouge_scorer


# Calculate the Rouge-L score between the target movie and all candidate movies
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) # Use Rouge-L scorer to calculate the similarity between the target movie and the candidate movies

def calculate_rougeL_score(target, candidates):
    scores = []
    for candidate in candidates:
        score = scorer.score(target, candidate)['rougeL'].fmeasure
        scores.append((candidate, score))
    return scores

def get_top_k_similar_movies(target_movie, candidate_movies, k=10):
    # Calculate the Rouge-L score between the target movie and all candidate movies
    scores = calculate_rougeL_score(target_movie, candidate_movies)

    # Sort the movies based on Rouge-L score in descending order and select the top k
    top_k_similar_movies = sorted(scores, key=lambda x: x[1], reverse=True)[:k]

    # Initialize two lists to store the top-k movies and their scores
    movies_list = [movie for movie, _ in top_k_similar_movies]
    scores_list = [score for _, score in top_k_similar_movies]

    return movies_list, scores_list

directory = 'ml-100k/'
direct_rec_processed = ['valid_2000_dir_k1_gpt4_processed.json', 'valid_2000_dir_k1_gpt35_processed.json']

with open(directory + "movie_taxonomy_5features.json", "r") as file:
    movies_dict = json.load(file)
all_candidate_movies = list(movies_dict.keys())


for processed_file in direct_rec_processed:
    with open(directory + processed_file, 'r') as file:
        generate = json.load(file)
        
    # next(iter(eval(generate[0]['output']).values()))[0]
    for index, entry in enumerate(generate):
        # print(index)
        output = entry['output'].strip('```').strip('json').replace('\"[', '[').replace(']\"', ']').replace('\\', '')
        top1_rec_movie = next(iter(eval(output).values()))[0]
        top10_rec_movies, top10_scores = get_top_k_similar_movies(top1_rec_movie, all_candidate_movies, k=10)
        entry['top10_similar_for_top1'] = top10_rec_movies
        entry['top10_scores'] = top10_scores

    # generate[0]['top10_similar_for_top1']
    with open(directory + processed_file, 'w') as file:
        json.dump(generate, file, indent=4)

In [141]:
import json

directory = 'ml-100k/'
with open(directory + 'valid_2000_tax_20feats_k10_gpt35.json', 'r') as file:
    generate = json.load(file)

# tax_generate = extract_rec_type(generate)
with open(directory + "movie_taxonomy_20features.json", "r") as file:
    movies_dict = json.load(file)

movies_dict_flatten = flatten_movies_dict(movies_dict)
generate_flatten = extract_rec_type(generate)

for entry in generate_flatten:
    rec_movies = []
    for rec_movie in entry['output']:
        max_common_features = 0
        for movie, movie_tax in movies_dict_flatten.items():
            try :
                common_features_count = len(set(rec_movie) & set(movie_tax))
            except:
                print(rec_movie)
                print(movie_tax)
            # common_features_count = len(set(rec_movie) & set(movie_tax))
            if common_features_count > max_common_features:
                max_common_features = common_features_count
                rec_movies.append(movie)
        if max_common_features == 0:
            rec_movies.append('No recommendation')
    entry['rec_movies'] = rec_movies

with open(directory + 'valid_2000_tax_20feats_k10_gpt35_processed.json', 'w') as file:
    json.dump(generate_flatten, file, indent=2)

In [74]:
def calculate_dcg(scores):
    """Calculate DCG for the given relevance scores."""
    import numpy as np
    return np.sum([
        (2**score - 1) / np.log2(idx + 2) for idx, score in enumerate(scores)
    ])

def calculate_ndcg_at_k(true_movie_name, movies_list, k):
    """Calculate NDCG@k for a given true movie name and a list of movie names."""
    # Assign relevance scores
    relevance_scores = [1 if name == true_movie_name else 0 for name in movies_list]
    
    # Calculate DCG@k
    dcg_at_k = calculate_dcg(relevance_scores[:k])
    
    # Calculate IDCG@k (ideal case where the true movie is at the top)
    idcg_at_k = calculate_dcg([1] + [0]*(k-1))
    
    # Calculate NDCG@k
    ndcg_at_k = dcg_at_k / idcg_at_k
    return ndcg_at_k

def calc_metrics_at_k(output, k=10):

    recall_count = 0  # Count of correct recommendations
    ndcg_score = 0  # Accumulated NDCG score

    for entry in output:
        recommended_movies = entry['top10_similar_for_top1']  # List of recommended movie names
        true_movie_name = entry['true_movie'].strip('"')  # True movie name

        if true_movie_name in recommended_movies[:k]:
            recall_count += 1
        
        # Calculate NDCG@K
        ndcg_score += calculate_ndcg_at_k(true_movie_name, recommended_movies, k)

    recall_at_k = recall_count / len(output)
    ndcg_at_k = ndcg_score / len(output)
    
    return recall_at_k, ndcg_at_k

In [142]:
# def calc_metrics_at_k(output, movie_dict, k=10):
#     from rouge_score import rouge_scorer
#     import numpy as np

#     recall_count = 0  # Count of correct recommendations
#     ndcg_score = 0  # Accumulated NDCG score
#     scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

#     for entry in output:
#         recommended_movies = entry['rec_movies']  # List of recommended movie names
#         true_movie_id = entry["true_movie_id"]
#         true_movie_name = movie_dict[true_movie_id] if true_movie_id in movie_dict else None

#         if not true_movie_name or not recommended_movies:
#             continue  # Skip if no true movie name or no recommendations
        
#         # Calculate relevance scores for each recommended movie
#         relevance_scores = []
#         for rec_movie_name in recommended_movies[:k]:
#             if rec_movie_name in movie_dict.values(): # to avoid "No recommendation"
#                 relevance_score = scorer.score(true_movie_name, rec_movie_name)['rouge1'].fmeasure
#             else:
#                 relevance_score = 0
#             relevance_scores.append(relevance_score)
        
#         # Calculate Recall@K
#         if any(score > 0 for score in relevance_scores):
#             recall_count += 1
        
#         # Calculate NDCG@K
#         dcg = 0
#         for i, score in enumerate(relevance_scores, start=1):
#             if score > 0:
#                 dcg += (2 ** score - 1) / np.log2(i + 1)
        
#         # Ideal DCG (IDCG) assumes the relevant item is at the first position
#         idcg = 1 / np.log2(1 + 1)
        
#         ndcg = dcg / idcg if dcg > 0 else 0
#         ndcg_score += ndcg

#     recall_at_k = recall_count / len(output)
#     ndcg_at_k = ndcg_score / len(output)
    
#     return recall_at_k, ndcg_at_k

In [48]:
# def calc_metrics_at_k(output, movie_dict, k=10):
#     from rouge_score import rouge_scorer
#     import numpy as np

#     recall_count = 0  # Count of correct recommendations
#     ndcg_score = 0  # Accumulated NDCG score
#     scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

#     for entry in output:
#         recommended_movies = entry['rec_movies']  # List of recommended movie names
#         true_movie_name = entry["true_movie"]
#         # true_movie_name = movie_dict[true_movie_id] if true_movie_id in movie_dict else None

#         if not true_movie_name or not recommended_movies:
#             continue  # Skip if no true movie name or no recommendations
        
#         # Calculate relevance scores for each recommended movie
#         relevance_scores = []
#         for rec_movie_name in recommended_movies[:k]:
#             if rec_movie_name in movie_dict.values(): 
#                 relevance_score = scorer.score(true_movie_name, rec_movie_name)['rougeL'].fmeasure
#             else:
#                 relevance_score = 0
#             relevance_scores.append(relevance_score)
        
#         # Calculate Recall@K
#         if any(score > 0.7 for score in relevance_scores):
#             recall_count += 1
        
#         # Calculate NDCG@K
#         dcg = 0
#         for i, score in enumerate(relevance_scores, start=1):
#             if score > 0:
#                 dcg += (2 ** score - 1) / np.log2(i + 1)
        
#         # Ideal DCG (IDCG) assumes the relevant item is at the first position
#         idcg = 1 / np.log2(1 + 1)
        
#         ndcg = dcg / idcg if dcg > 0 else 0
#         ndcg_score += ndcg

#     recall_at_k = recall_count / len(output)
#     ndcg_at_k = ndcg_score / len(output)
    
#     return recall_at_k, ndcg_at_k

In [143]:
# def acc_truth_rouge1(output, movie_dict):
#     from rouge_score import rouge_scorer
    
#     correct = 0
#     exist = 0
#     scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

#     for entry in output:
#         if entry['rec_movie']:
#             generated_movie = entry["rec_movie"]
#         else:
#             generated_movie = None
#         true_movie_id = entry["true_movie_id"]

#         # Initialize variables to store the best score and corresponding movie ID
#         best_score = 0
#         best_movie_id = None

#         for movie_id, movie_name in movie_dict.items():
#             if generated_movie == None: # If the model fails to generate a movie
#                 score = 0
#             else:
#                 scores = scorer.score(movie_name, generated_movie)
#                 score = scores['rouge1'].fmeasure

#             if score > best_score:
#                 best_score = score
#                 best_movie_id = movie_id

#         # Check if the best matching movie ID is the correct one
#         if best_movie_id == true_movie_id:
#             correct += 1
#         if best_score > 0.95:
#         # if best_score == 1:
#             exist += 1
    
#     return correct / len(output), exist / len(output)

In [22]:
# def calc_metrics_at_k(output, movie_dict, k=10):
#     from rouge_score import rouge_scorer
#     import numpy as np

#     recall_count = 0  # Count of correct recommendations
#     ndcg_score = 0  # Accumulated NDCG score
#     scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

#     for entry in output:
#         recommended_movies = entry['top10_similar_for_top1']  # List of recommended movie names
#         true_movie_name = entry["true_movie"]
#         # true_movie_name = movie_dict[true_movie_id] if true_movie_id in movie_dict else None

#         if not true_movie_name or not recommended_movies:
#             continue  # Skip if no true movie name or no recommendations
        
#         # Calculate relevance scores for each recommended movie
#         relevance_scores = []
#         for rec_movie_name in recommended_movies[:k]:
#             # relevance_score = scorer.score(true_movie_name, rec_movie_name)['rougeL'].fmeasure
#             if rec_movie_name in movie_dict.values(): 
#                 relevance_score = scorer.score(true_movie_name, rec_movie_name)['rougeL'].fmeasure
#             else:
#                 relevance_score = 0
#             relevance_scores.append(relevance_score)
        
#         # Calculate Recall@K
#         if any(score > 0.6 for score in relevance_scores):
#             recall_count += 1
        
#         # Calculate NDCG@K
#         dcg = 0
#         for i, score in enumerate(relevance_scores, start=1):
#             if score > 0:
#                 dcg += (2 ** score - 1) / np.log2(i + 1)
        
#         # Ideal DCG (IDCG) assumes the relevant item is at the first position
#         idcg = 1 / np.log2(1 + 1)
        
#         ndcg = dcg / idcg if dcg > 0 else 0
#         ndcg_score += ndcg

#     recall_at_k = recall_count / len(output)
#     ndcg_at_k = ndcg_score / len(output)
    
#     return recall_at_k, ndcg_at_k

In [8]:
file_name = 'ml-100k/valid_2000_tax_5feats_k1_gpt35_processed.json'
with open(file_name, 'r') as file:
    results = json.load(file)

results[0]['true_movie'].strip('"')
# for entry in output:
#     recommended_movies = entry['top10_similar_for_top1']  # List of recommended movie names
#     true_movie_name = entry['true_movie']  # True movie name

#     if true_movie_name in recommended_movies[:k]:
#         recall_count += 1

'Raising Arizona (1987)'

In [10]:
import json

directory = 'ml-100k/'
direct_rec = [directory + 'valid_2000_dir_k10_gpt4_processed.json', 
              directory + 'valid_2000_dir_k10_gpt35_processed.json']

tax_rec = [directory + 'valid_2000_tax_5feats_k10_gpt4_processed.json', 
           directory + 'valid_2000_tax_5feats_k10_gpt35_processed.json',
           directory + 'valid_2000_tax_20feats_k10_gpt4_processed.json', 
           directory + 'valid_2000_tax_20feats_k10_gpt35_processed.json']

# -------------------------------------------------------

gpt_35_res = [directory + 'valid_2000_dir_k1_gpt35_processed.json',
              directory + 'valid_2000_tax_5feats_k1_gpt35_processed.json',
              directory + 'valid_2000_tax_20feats_k1_gpt35_processed.json']

gpt_4_res = [directory + 'valid_2000_dir_k1_gpt4_processed.json',
             directory + 'valid_2000_tax_5feats_k1_gpt4_processed.json',
             directory + 'valid_2000_tax_20feats_k1_gpt4_processed.json']

# gpt_35_res = [directory + 'valid_2000_dir_k10_gpt35_processed.json',
#               directory + 'valid_2000_tax_5feats_k10_gpt35_top1.json',
#               directory + 'valid_2000_tax_20feats_k10_gpt35_top1.json']

# gpt_4_res = [directory + 'valid_2000_dir_k10_gpt4_processed.json',
#              directory + 'valid_2000_tax_5feats_k10_gpt4_top1.json',
#              directory + 'valid_2000_tax_20feats_k10_gpt4_top1.json']

# file_list = direct_rec + tax_rec
file_list = gpt_35_res + gpt_4_res


for k in [1, 5, 10]:
    print(f"@{k}")
    for file in file_list:
        print(file.strip('_processed.json').split('2000_')[1])
        with open(file, 'r') as file:
            results = json.load(file)

        f = open(directory + 'u.item', 'r', encoding='ISO-8859-1')
        movies = f.readlines()

        movie_names = [_.split('|')[1] for _ in movies]
        movie_ids = [_.split('|')[0] for _ in movies]
        movie_dict = dict(zip(movie_ids, movie_names))

        # recall_at_k, ndcg_at_k = calc_metrics_at_k(results, movie_dict, k=k)
        recall_at_k, ndcg_at_k = calc_metrics_at_k(results, k=k)
        print(f"Recall@{k}: {recall_at_k:.4f}, NDCG@{k}: {ndcg_at_k:.4f}")
    print('------------------------------------------------------')

@1
dir_k1_gpt35
Recall@1: 0.0110, NDCG@1: 0.0110
tax_5feats_k1_gpt35
Recall@1: 0.0020, NDCG@1: 0.0020
tax_20feats_k1_gpt35
Recall@1: 0.0015, NDCG@1: 0.0015
dir_k1_gpt4
Recall@1: 0.0045, NDCG@1: 0.0045
tax_5feats_k1_gpt4
Recall@1: 0.0035, NDCG@1: 0.0035
tax_20feats_k1_gpt4
Recall@1: 0.0020, NDCG@1: 0.0020
------------------------------------------------------
@5
dir_k1_gpt35
Recall@5: 0.0190, NDCG@5: 0.0148
tax_5feats_k1_gpt35
Recall@5: 0.0085, NDCG@5: 0.0051
tax_20feats_k1_gpt35
Recall@5: 0.0155, NDCG@5: 0.0083
dir_k1_gpt4
Recall@5: 0.0100, NDCG@5: 0.0074
tax_5feats_k1_gpt4
Recall@5: 0.0135, NDCG@5: 0.0081
tax_20feats_k1_gpt4
Recall@5: 0.0170, NDCG@5: 0.0091
------------------------------------------------------
@10
dir_k1_gpt35
Recall@10: 0.0285, NDCG@10: 0.0178
tax_5feats_k1_gpt35
Recall@10: 0.0185, NDCG@10: 0.0084
tax_20feats_k1_gpt35
Recall@10: 0.0235, NDCG@10: 0.0109
dir_k1_gpt4
Recall@10: 0.0180, NDCG@10: 0.0099
tax_5feats_k1_gpt4
Recall@10: 0.0255, NDCG@10: 0.0119
tax_20feats_k1

In [75]:
import json

directory = 'ml-100k/'

gpt_35_name = [directory + 'valid_2000_dir_k1_gpt35_processed.json',
              directory + 'valid_2000_tax_5feats_name_k1_gpt35_processed.json',
              directory + 'valid_2000_tax_20feats_name_k1_gpt35_processed.json']

gpt_4_name = [directory + 'valid_2000_dir_k1_gpt4_processed.json',
             directory + 'valid_2000_tax_5feats_name_k1_gpt4_processed.json',
             directory + 'valid_2000_tax_20feats_name_k1_gpt4_processed.json']

file_list = gpt_35_name + gpt_4_name


for k in [1, 5, 10]:
    print(f"@{k}")
    for file in file_list:
        print(file.strip('_processed.json').split('2000_')[1])
        with open(file, 'r') as file:
            results = json.load(file)

        f = open(directory + 'u.item', 'r', encoding='ISO-8859-1')
        movies = f.readlines()

        movie_names = [_.split('|')[1] for _ in movies]
        movie_ids = [_.split('|')[0] for _ in movies]
        movie_dict = dict(zip(movie_ids, movie_names))

        # recall_at_k, ndcg_at_k = calc_metrics_at_k(results, movie_dict, k=k)
        recall_at_k, ndcg_at_k = calc_metrics_at_k(results, k=k)
        print(f"Recall@{k}: {recall_at_k:.4f}, NDCG@{k}: {ndcg_at_k:.4f}")
    print('------------------------------------------------------')

@1
dir_k1_gpt35
Recall@1: 0.0110, NDCG@1: 0.0110
tax_5feats_name_k1_gpt35
Recall@1: 0.0040, NDCG@1: 0.0040
tax_20feats_name_k1_gpt35
Recall@1: 0.0040, NDCG@1: 0.0040
dir_k1_gpt4
Recall@1: 0.0045, NDCG@1: 0.0045
tax_5feats_name_k1_gpt4
Recall@1: 0.0060, NDCG@1: 0.0060
tax_20feats_name_k1_gpt4
Recall@1: 0.0055, NDCG@1: 0.0055
------------------------------------------------------
@5
dir_k1_gpt35
Recall@5: 0.0190, NDCG@5: 0.0148
tax_5feats_name_k1_gpt35
Recall@5: 0.0120, NDCG@5: 0.0081
tax_20feats_name_k1_gpt35
Recall@5: 0.0120, NDCG@5: 0.0078
dir_k1_gpt4
Recall@5: 0.0100, NDCG@5: 0.0074
tax_5feats_name_k1_gpt4
Recall@5: 0.0115, NDCG@5: 0.0086
tax_20feats_name_k1_gpt4
Recall@5: 0.0140, NDCG@5: 0.0095
------------------------------------------------------
@10
dir_k1_gpt35
Recall@10: 0.0285, NDCG@10: 0.0178
tax_5feats_name_k1_gpt35
Recall@10: 0.0195, NDCG@10: 0.0104
tax_20feats_name_k1_gpt35
Recall@10: 0.0225, NDCG@10: 0.0111
dir_k1_gpt4
Recall@10: 0.0180, NDCG@10: 0.0099
tax_5feats_name_k1